In [1]:
import numpy as np
import pandas as pd
import hvplot.pandas

In [2]:
# Set the random seed for reproducibility
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [3]:
# Load the fear and greed sentiment data for Bitcoin
df = pd.read_csv('data_fng_coin.csv')
df.head()

,fng_value,Close
0,81,3703.229492
1,84,3705.705322
2,78,3593.494385
3,77,3579.811523
4,75,3657.249268


In [4]:
# It returns a numpy array of X any y
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [5]:
# Predict Closing Prices using a 1 day window of previous closing prices
window_size = 1

# Column index 0 is the 'fng_value' column
# Column index 1 is the `Close` column
feature_column = 1
target_column = 1
X, y = window_data(df, window_size, feature_column, target_column)

In [6]:
X_train = X
X_test = X
y_train = y
y_test = y

In [7]:
# Use the MinMaxScaler to scale data between 0 and 1.
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X)

MinMaxScaler()

In [8]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

scaler.fit(y)

y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)

In [9]:
# Reshape the features for the model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [11]:
# Define the LSTM RNN model.
model = Sequential()

number_units = 50
dropout_fraction = 0.2

# Layer 1
model.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train.shape[1], 1))
    )
model.add(Dropout(dropout_fraction))
# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))
# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))
# Output layer
model.add(Dense(1))

In [12]:
# Compile the model
model.compile(optimizer="adam", loss="mean_squared_error")

In [13]:
# Summarize the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 50)             10400     
                                                                 
 dropout (Dropout)           (None, 1, 50)             0         
                                                                 
 lstm_1 (LSTM)               (None, 1, 50)             20200     
                                                                 
 dropout_1 (Dropout)         (None, 1, 50)             0         
                                                                 
 lstm_2 (LSTM)               (None, 50)                20200     
                                                                 
 dropout_2 (Dropout)         (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 5

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=15, shuffle=False, batch_size=1, verbose=1)

Epoch 1/15
698/698 [==============================] - 23s 20ms/step - loss: 0.0244
Epoch 2/15
698/698 [==============================] - 14s 20ms/step - loss: 0.0114
Epoch 3/15
698/698 [==============================] - 13s 19ms/step - loss: 0.0105
Epoch 4/15
698/698 [==============================] - 12s 18ms/step - loss: 0.0086
Epoch 5/15
698/698 [==============================] - 14s 20ms/step - loss: 0.0066
Epoch 6/15
698/698 [==============================] - 13s 19ms/step - loss: 0.0048
Epoch 7/15
698/698 [==============================] - 12s 18ms/step - loss: 0.0041
Epoch 8/15
698/698 [==============================] - 14s 20ms/step - loss: 0.0034
Epoch 9/15
698/698 [==============================] - 13s 19ms/step - loss: 0.0035
Epoch 10/15
698/698 [==============================] - 13s 19ms/step - loss: 0.0036
Epoch 11/15
698/698 [==============================] - 13s 19ms/step - loss: 0.0038
Epoch 12/15
698/698 [==============================] - 14s 20ms/step - loss: 0.0041
E

## Model Performance

In [27]:
# Evaluate the model
model.evaluate(X_test, y_test)
model.save("lstm_model.h5")

22/22 [==============================] - 3s 12ms/step - loss: 0.0155


In [28]:
model

In [48]:
# Make some predictions
predicted = model.predict(X_test)

22/22 [==============================] - 0s 9ms/step


In [49]:
# Recover the original prices instead of the scaled version
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

In [50]:
# Create a DataFrame of Real and Predicted values
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
}, index = df.index[-len(real_prices):]) 
stocks.head()

,Real,Predicted
2024-11-24,3413.543945,2935.364502
2024-11-23,3363.659912,2999.673340
2024-11-22,3396.223389,2963.072266
2024-11-21,3331.600830,2987.049561
2024-11-20,3361.053955,2939.177734


In [51]:
stocks.hvplot()

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
import streamlit as st

# Hàm tải và xử lý dữ liệu
def load_data(file_path):
    data = pd.read_csv(file_path)
    data = data['Close'].values.reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)
    return scaled_data, scaler

# Chuẩn bị tập dữ liệu cho LSTM
def prepare_data(data, time_step=60):
    X, y = [], []
    for i in range(time_step, len(data)):
        X.append(data[i-time_step:i, 0])
        y.append(data[i, 0])
    X = np.array(X)
    y = np.array(y)
    return X.reshape(X.shape[0], X.shape[1], 1), y

# Xây dựng mô hình LSTM
def build_lstm(input_shape):
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Ứng dụng Streamlit
st.title("Dự đoán giá cổ phiếu bằng LSTM")
uploaded_file = st.file_uploader("bitcoin_historic", type=["csv"])

if uploaded_file is not None:
    # Xử lý dữ liệu
    scaled_data, scaler = load_data(uploaded_file)
    train_size = int(len(scaled_data) * 0.8)
    train_data, test_data = scaled_data[:train_size], scaled_data[train_size:]
    time_step = 60
    X_train, y_train = prepare_data(train_data, time_step)
    X_test, y_test = prepare_data(test_data, time_step)

    # Huấn luyện mô hình
    st.write("Đang huấn luyện mô hình...")
    model = build_lstm((X_train.shape[1], 1))
    model.fit(X_train, y_train, batch_size=32, epochs=5, verbose=1)

    # Dự đoán
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions.reshape(-1, 1))
    y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Hiển thị biểu đồ
    st.subheader("Kết quả dự đoán")
    plt.figure(figsize=(12, 6))
    plt.plot(y_test_actual, label="Thực tế")
    plt.plot(predictions, label="Dự đoán")
    plt.legend()
    st.pyplot(plt)

2024-11-29 22:06:27.716 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-29 22:06:30.205 
  command:

    streamlit run D:\anaconda3\envs\pygpu\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-11-29 22:06:30.207 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-29 22:06:30.209 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-29 22:06:30.210 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-29 22:06:30.211 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-29 22:06:30.221 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-29 22:06:30.223 Thread 'MainThread': mis